In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczem49.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema148.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema48.jpeg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema.8.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema149.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema-acute-22.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema-chronic-47.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema54.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema8.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema28..jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema-lids-9.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema134.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema53.jpg
/kaggle/input/face-skin-disease/DATA/testing/Eczemaa/eczema155.png
/kaggle/input/face-skin-disease/DATA/testing/Eczema

In [1]:
# STEP 1: Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
import cv2
from PIL import Image
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img, save_img
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

2025-06-17 18:47:46.816744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750186067.005257      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750186067.061929      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# STEP 2: Load Image Paths and Labels
train_df = pd.DataFrame(columns=["image_path", "label"])
train_dir = "/kaggle/input/face-skin-disease/DATA/train"

for label in os.listdir(train_dir):
    for img_file in os.listdir(os.path.join(train_dir, label)):
        train_df.loc[len(train_df)] = [os.path.join(train_dir, label, img_file), label]

train_df.dropna(inplace=True)

In [3]:
import os
import pandas as pd

dataset_path = '/kaggle/input/selfies-id-images-dataset/Selfies ID Images dataset'
data = []

for group in os.listdir(dataset_path):
    group_path = os.path.join(dataset_path, group)
    
    if not os.path.isdir(group_path):
        continue
    
    for person_folder in os.listdir(group_path):
        person_path = os.path.join(group_path, person_folder)
        
        if not os.path.isdir(person_path):
            continue
        
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            
            # Optionally filter for image file extensions
            if not image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                continue
            
            data.append({
                'image_path': image_path,
                'label': 'normal'
            })

# Create DataFrame
df = pd.DataFrame(data)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# Show sample
print(df.head())
print(f"\nTotal image paths collected: {len(df)}")


                                          image_path   label
0  /kaggle/input/selfies-id-images-dataset/Selfie...  normal
1  /kaggle/input/selfies-id-images-dataset/Selfie...  normal
2  /kaggle/input/selfies-id-images-dataset/Selfie...  normal
3  /kaggle/input/selfies-id-images-dataset/Selfie...  normal
4  /kaggle/input/selfies-id-images-dataset/Selfie...  normal

Total image paths collected: 435


In [4]:
for i in range(0,236):
    train_df.loc[len(train_df)] = [df.iloc[i][0], df.iloc[i][1]]

In [5]:
train_df['label'].value_counts()

label
normal                  236
Eczemaa                  88
Acne                     88
Rosacea                  88
Actinic Keratosis        88
Basal Cell Carcinoma     88
Name: count, dtype: int64

In [6]:
# STEP 3: Encode Labels
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:")
for k, v in label_mapping.items():
    print(f"{v:2} → {k}")

Label Mapping:
 0 → Acne
 1 → Actinic Keratosis
 2 → Basal Cell Carcinoma
 3 → Eczemaa
 4 → Rosacea
 5 → normal


In [7]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

from albumentations import (
    HorizontalFlip, VerticalFlip, ShiftScaleRotate, RandomBrightnessContrast,
    GaussNoise, ElasticTransform, GridDistortion, CLAHE, OneOf, Compose
)
from albumentations.augmentations.dropout import CoarseDropout  # ✅ Correct import

# Parameters
target_count = 500
output_dir = 'albumented_images'
img_size = (224, 224)
os.makedirs(output_dir, exist_ok=True)

# Albumentations Transform Pipeline
augment = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.3),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),
    RandomBrightnessContrast(p=0.5),
    GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    OneOf([
        ElasticTransform(p=0.2, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        GridDistortion(p=0.2),
    ], p=0.3),
    CLAHE(p=0.2),
    CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.4),
])

# Load class distribution
label_counts = train_df['label'].value_counts().to_dict()
new_image_paths = []
new_labels = []

# Augment underrepresented classes
for label in tqdm(label_counts, desc="Albumenting underrepresented classes"):
    paths = train_df[train_df['label'] == label]['image_path'].values
    count = len(paths)

    if count >= target_count:
        continue

    required = target_count - count
    print(f"\n[INFO] Albumenting class '{label}' from {count} to {target_count}...")

    i = 0
    while i < required:
        path = paths[i % count]
        try:
            img = cv2.imread(path)
            if img is None:
                print(f"[WARNING] Could not read image: {path}")
                i += 1
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)

            augmented = augment(image=img)
            aug_img = augmented['image']

            save_path = os.path.join(output_dir, f"{label}_{i}.jpg")
            cv2.imwrite(save_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

            new_image_paths.append(save_path)
            new_labels.append(label)
            i += 1
        except Exception as e:
            print(f"[ERROR] Failed to process {path}: {e}")
            i += 1

# Final balanced dataframe
aug_df = pd.DataFrame({'image_path': new_image_paths, 'label': new_labels})
balanced_df = pd.concat([train_df, aug_df], ignore_index=True)

# Save for reproducibility
balanced_df.to_csv("balanced_train_df.csv", index=False)

# Summary
print("\n✅ [INFO] Class distribution after Albumentation:")
print(balanced_df['label'].value_counts())


Albumenting underrepresented classes:   0%|          | 0/6 [00:00<?, ?it/s]


[INFO] Albumenting class '5' from 236 to 500...


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Albumenting underrepresented classes:  17%|█▋        | 1/6 [00:28<02:24, 28.89s/it]


[INFO] Albumenting class '3' from 88 to 500...


Albumenting underrepresented classes:  33%|███▎      | 2/6 [00:33<00:57, 14.39s/it]

...


[INFO] Albumenting class '1' from 88 to 500...


Albumenting underrepresented classes:  83%|████████▎ | 5/6 [00:45<00:06,  6.31s/it]


[INFO] Albumenting class '2' from 88 to 500...


Albumenting underrepresented classes: 100%|██████████| 6/6 [00:49<00:00,  8.31s/it]


✅ [INFO] Class distribution after Albumentation:
label
3    500
0    500
4    500
1    500
2    500
5    500
Name: count, dtype: int64


In [8]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet101, DenseNet121
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [9]:
# STEP 5: Train-Validation Split
train_df_1, val_df = train_test_split(
    balanced_df,
    test_size=0.2,
    stratify=balanced_df['label'],
    random_state=42
)

train_df_1['label'] = train_df_1['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)

In [10]:
# STEP 6: ImageDataGenerators
train_gen = ImageDataGenerator(preprocessing_function=resnet_preprocess)
val_gen = ImageDataGenerator(preprocessing_function=resnet_preprocess)

img_size = (224, 224)
batch_size = 16

train_generator = train_gen.flow_from_dataframe(
    train_df_1,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_gen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

num_classes = balanced_df['label'].nunique()


Found 2400 validated image filenames belonging to 6 classes.
Found 600 validated image filenames belonging to 6 classes.


In [13]:
from keras.applications import ResNet101, DenseNet121
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.layers import Concatenate, Multiply, Add, Lambda
from keras.models import Model
from keras.optimizers import Adam
import tensorflow as tf

# Define input
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

# ResNet101 branch
resnet_base = ResNet101(include_top=False, weights='imagenet', input_shape=input_shape)
x = resnet_base(input_tensor)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)

# DenseNet121 branch
densenet_base = DenseNet121(include_top=False, weights='imagenet', input_shape=input_shape)
y = densenet_base(input_tensor)
y = GlobalAveragePooling2D()(y)
y = Dense(512, activation='relu')(y)
y = BatchNormalization()(y)
y = Dense(256, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)
y = Dense(128, activation='relu')(y)

# Attention fusion
concat = Concatenate()([x, y])  # shape (batch_size, 256)
attn_weights = Dense(2, activation='softmax')(concat)  # shape (batch_size, 2)

alpha_x = Lambda(lambda z: tf.expand_dims(z[:, 0], axis=-1))(attn_weights)  # (batch_size, 1)
alpha_y = Lambda(lambda z: tf.expand_dims(z[:, 1], axis=-1))(attn_weights)  # (batch_size, 1)

x_scaled = Multiply()([x, alpha_x])
y_scaled = Multiply()([y, alpha_y])

combined = Add()([x_scaled, y_scaled])

# Additional layers after fusion
combined = Dense(256, activation='relu')(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

combined = Dense(128, activation='relu')(combined)
combined = BatchNormalization()(combined)
combined = Dropout(0.3)(combined)

# Final output layer (adjust classes as needed)
output = Dense(6, activation='softmax')(combined)

# Build model
model = Model(inputs=input_tensor, outputs=output)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet101 (Functional)    │ (None, 7, 7, 2048)     │     42,658,176 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ densenet121 (Functional)  │ (None, 7, 7, 1024)     │      7,037,504 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ resnet101[0][0]        │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1024)           │              0 │ densenet121[0][0]      │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 512)            │      1,049,088 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 512)            │        524,800 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_6     │ (None, 512)            │          2,048 │ dense_10[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_8     │ (None, 512)            │          2,048 │ dense_13[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 256)            │        131,328 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 256)            │        131,328 │ batch_normalization_8… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_7     │ (None, 256)            │          1,024 │ dense_11[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_9     │ (None, 256)            │          1,024 │ dense_14[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 256)            │              0 │ batch_normalization_7… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 256)            │              0 │ batch_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 128)            │         32,896 │ dropout_4[0][0]        │
├──────────────────────

 Total params: 51,672,904 (197.12 MB)

 Trainable params: 51,480,072 (196.38 MB)

 Non-trainable params: 192,832 (753.25 KB)

In [14]:
# STEP 8: Train the Model
callbacks = [
    ModelCheckpoint('imagenet_resnet101_best_model.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    #callbacks=callbacks
)

Epoch 1/25


I0000 00:00:1750186972.526623     103 service.cc:148] XLA service 0x7eab84002c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750186972.527333     103 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750186972.527354     103 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750186991.093287     103 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750187116.765475     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


150/150 ━━━━━━━━━━━━━━━━━━━━ 449s 760ms/step - accuracy: 0.2471 - loss: 2.2785 - val_accuracy: 0.3667 - val_loss: 1.7280
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 77s 510ms/step - accuracy: 0.3592 - loss: 1.9055 - val_accuracy: 0.4833 - val_loss: 1.3102
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 501ms/step - accuracy: 0.4576 - loss: 1.5676 - val_accuracy: 0.4867 - val_loss: 1.5770
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 499ms/step - accuracy: 0.4575 - loss: 1.5611 - val_accuracy: 0.5067 - val_loss: 1.3835
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 502ms/step - accuracy: 0.4881 - loss: 1.4496 - val_accuracy: 0.6100 - val_loss: 1.0806
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 76s 502ms/step - accuracy: 0.5141 - loss: 1.3584 - val_accuracy: 0.5900 - val_loss: 1.1300
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 503ms/step - accuracy: 0.5695 - loss: 1.1760 - val_accuracy: 0.6000 - val_loss: 1.1535
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 500ms/step - accuracy: 0.5802 - loss: 1.1475 - va

In [15]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    #callbacks=callbacks
)

Epoch 1/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 76s 509ms/step - accuracy: 0.9176 - loss: 0.2536 - val_accuracy: 0.8017 - val_loss: 0.7412
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 501ms/step - accuracy: 0.9217 - loss: 0.2519 - val_accuracy: 0.8450 - val_loss: 0.5493
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 498ms/step - accuracy: 0.9321 - loss: 0.2090 - val_accuracy: 0.8300 - val_loss: 0.5639
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 502ms/step - accuracy: 0.9256 - loss: 0.2265 - val_accuracy: 0.8017 - val_loss: 0.6891
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 499ms/step - accuracy: 0.9254 - loss: 0.2289 - val_accuracy: 0.8367 - val_loss: 0.5219
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 497ms/step - accuracy: 0.9360 - loss: 0.2194 - val_accuracy: 0.8717 - val_loss: 0.4343
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 500ms/step - accuracy: 0.9524 - loss: 0.1631 - val_accuracy: 0.8333 - val_loss: 0.5169
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 75s 499ms/step - accuracy: 0.9355 - loss: 0

In [16]:
# STEP 9: Evaluate on Validation Set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.8436 - loss: 0.5004
Validation Accuracy: 0.8500


In [17]:
# STEP 10: Load and Predict on Test Set

# Prepare test dataframe
test_df = pd.DataFrame(columns=["image_path", "label"])
test_dir = "/kaggle/input/face-skin-disease/DATA/testing"

# Load test image paths and labels
for label in os.listdir(test_dir):
    for img in os.listdir(os.path.join(test_dir, label)):
        test_df.loc[len(test_df)] = [os.path.join(test_dir, label, img), label]

In [18]:
for i in range(len(df)-38,len(df)):
    test_df.loc[len(test_df)] = [df.iloc[i][0], df.iloc[i][1]]

In [20]:
test_df['label'].value_counts()

label
normal                  38
Eczemaa                 37
Acne                    37
Rosacea                 37
Actinic Keratosis       37
Basal Cell Carcinoma    37
Name: count, dtype: int64

In [21]:
# Keep only known labels from training
known_labels = set(label_encoder.classes_)
test_df = test_df[test_df['label'].isin(known_labels)].reset_index(drop=True)

# Encode labels
test_df['label'] = label_encoder.transform(test_df['label'])
test_df['label'] = test_df['label'].astype(str)

# Create test data generator
test_generator = val_gen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")

Found 223 validated image filenames belonging to 6 classes.
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8348 - loss: 0.7942 
✅ Test Accuracy: 0.8475
